In [18]:
import sys, os
parent_directory = os.path.dirname(os.getcwd())
sys.path.append(parent_directory)
import tool

import scipy
import pandas as pd
import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectMultiple, fixed

import warnings
warnings.filterwarnings('ignore')

In [19]:
"""
Data initialisation
"""

asset_index = pd.read_csv("../data/AIDX.csv")

num_limit = np.random.randint(*(5,10))
index_list = asset_index['S_IRDCODE'].drop_duplicates().sample(num_limit).tolist()

asset_index['TRADE_DT'] = pd.to_datetime(asset_index['TRADE_DT'], format='%Y%m%d')
asset_index.sort_values(by='TRADE_DT', inplace=True)
asset_index.set_index('TRADE_DT', inplace=True)
asset_index = asset_index.pivot(columns='S_IRDCODE', values='CLOSE').ffill()[index_list].dropna()

print(index_list)
print(asset_index)

['931580HKD01.CSI', '399646.SZ', 'h20087.CSI', '921382.CSI', '931170.CSI']
S_IRDCODE   931580HKD01.CSI  399646.SZ  h20087.CSI  921382.CSI  931170.CSI
TRADE_DT                                                                  
2023-01-03         562.3470  8438.5202   4277.9228   1540.4712   1962.6112
2023-01-04         578.3944  8506.5579   4254.8251   1538.3449   1977.2365
2023-01-05         587.6140  8756.7102   4305.7353   1573.7741   1998.4170
2023-01-06         578.1793  8715.0140   4325.0404   1585.6788   2007.1215
2023-01-09         597.3721  8837.8333   4347.7910   1601.0404   2031.3875
...                     ...        ...         ...         ...         ...
2023-12-20         546.0745  7657.6932   3851.5766   1458.5027   1961.6502
2023-12-21         546.0745  7657.6932   3851.5766   1458.5027   1961.6502
2023-12-22         546.0745  7657.6932   3851.5766   1458.5027   1961.6502
2023-12-25         546.0745  7657.6932   3851.5766   1458.5027   1961.6502
2023-12-26         546.07

In [20]:
"""
Parameters
"""

BACKTEST_DAY = 30
MODEL_TYPE = 'BL' # MVO, RP, BL, RB
TARGET_RETURN = 0.0 # target return
RISK_FREE_RATE = 0.02 # risk-free rate
REBALANCE_DAYS = 30

N = len(index_list)
index_min_weight = [0 for _ in range(N)]
index_max_weight = [1 for _ in range(N)]
WEIGHT_CONSTRAINTS = list(zip(index_min_weight, index_max_weight))

In [21]:
"""
Rebalancing
"""

def rebalance(asset_index, T):
    predicts = []
    actuals = []
    realities = []
    
    for i in range(T, len(asset_index), T):
        
        if i+T >= len(asset_index):
            break
        
        historical_data = asset_index[i-T:i]
        future_data = asset_index[i:i+T]
        
        predict, actual = tool.evaluate(historical_data, future_data, WEIGHT_CONSTRAINTS, MODEL_TYPE, TARGET_RETURN, RISK_FREE_RATE)
        predicts.append(predict)
        actuals.append(actual)
        
        reality = tool.check([1 / N for _ in range(N)], future_data, RISK_FREE_RATE)  # equally weighed
        realities.append(reality)
    
    return predicts, actuals, realities

predicts, actuals, realities = rebalance(asset_index, REBALANCE_DAYS)
print(predicts)
print(actuals)
print(realities)

[(0.9322765431966051, 86.97835949144091, 0.010488546214606145), (-0.2543072521177438, 59.8105698383647, -0.004586267157444687), (-0.21838506934927734, 137.17898963059565, -0.0017377666214863944), (0.17845116393365168, 41.447534802219494, 0.003822933370820566), (0.03527865209486822, 38.00346033721919, 0.0004020331822232743), (-0.416222557211081, 93.35914132638234, -0.004672521094490925), (-0.18971945822526642, 22.795598520460718, -0.009199997887181064)]
[(-0.03844692725028062, 54.8405870528287, -0.0010657604229141072), (-0.098671208202123, 73.65057794868225, -0.001611273278599523), (0.13490283370046316, 153.58264397467468, 0.0007481498607317263), (0.15795711245288582, 32.38654651644494, 0.004259704330711373), (-0.416222557211081, 93.35914132638234, -0.004672521094490925), (-0.3582529343495715, 69.85709575595783, -0.0054146673327356565), (-4.136304873467055e-23, 2.67132060876011e-13, -74869335917.27492)]
[(0.11389418065525589, 68.3200431084278, 0.0013743284749724246), (0.1523983046632845

In [22]:
"""
Output
"""

def display(L1, L2, L3, normalize=False, lines_to_show=None):
    line_names = ['r1', 'v1', 's1', 'r2', 'v2', 's2', 'r3', 'v3', 's3']
    line_styles = {
        'r1': 'r-', 'v1': 'r--', 's1': 'r:',
        'r2': 'b-', 'v2': 'b--', 's2': 'b:',
        'r3': 'g-', 'v3': 'g--', 's3': 'g:'
    }

    a1, b1, c1 = zip(*L1)
    a2, b2, c2 = zip(*L2)
    a3, b3, c3 = zip(*L3)

    lines = {'r1': a1, 'v1': b1, 's1': c1, 
             'r2': a2, 'v2': b2, 's2': c2,
             'r3': a3, 'v3': b3, 's3': c3}

    def normalize_data(data):
        return (data - np.mean(data)) / np.std(data)

    if normalize:
        lines = {name: normalize_data(data) for name, data in lines.items()}

    plt.figure(figsize=(10, 6))
    for line in lines_to_show:
        plt.plot(lines[line], line_styles[line], label=line, marker='o')
    
    plt.title("R: Return\tV: Volatility\tS: Sharpe Ratio\n1: Predicted\t2: Actual\t3: Reality (Equally Weighed)\nButton 'normalize': Normalise each line")
    plt.legend()
    
    for pair in [('r1', 'r2'), ('v1', 'v2'), ('s1', 's2')]:
        if pair[0] in lines_to_show and pair[1] in lines_to_show:
            corr, _ = scipy.stats.spearmanr(lines[pair[0]], lines[pair[1]])
            print(f"Spearman correlation between {pair[0]} and {pair[1]}: {corr:.2f}")
    
    plt.show()


# @interact
interact(display, 
         L1=fixed(predicts), 
         L2=fixed(actuals),
         L3=fixed(realities), 
         normalize=True, 
         lines_to_show=SelectMultiple(options=['r1', 'r2', 'r3', 'v1', 'v2', 'v3', 's1', 's2', 's3'],
                                      value=['r1', 'r2', 'r3', 'v1', 'v2', 'v3', 's1', 's2', 's3'], 
                                      description='Lines'))



interactive(children=(Checkbox(value=True, description='normalize'), SelectMultiple(description='Lines', index…

<function __main__.display(L1, L2, L3, normalize=False, lines_to_show=None)>